In [3]:
import definitions as df
import data_manipulation as dm
import sql_def as sql
import os
import numpy as np
import temp.File_Def as FD
import matplotlib.pyplot as plt
import plots
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import math
import torch.nn.functional as F

Grabing the data from SQL dataset

In [4]:
path_to_database = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project\data"
path_to_code = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project"
os.chdir(path_to_database)
track_from_sql =  sql.data_from_sql("database.db", "SELECT * FROM TRACKMATEDATA LIMIT 7000")
os.chdir(path_to_code)
track_with_frame = df.separate_trajectories(track_from_sql)
trajectories = df.separate_data(track_with_frame, False)
len(trajectories)

473

Making Fake data

In [5]:
total_size = 1000
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 5

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)
trajectories

[[(767.0, 641.0),
  (773.973363, 641.61009),
  (780.946726, 642.22018),
  (787.920089, 642.830271),
  (794.893452, 643.440361)],
 [(549, 26), (555, 26), (561, 26), (567, 26), (573, 26)],
 [(164.0, 988.0),
  (171.922145, 989.113385),
  (179.844289, 990.22677),
  (187.766434, 991.340154),
  (195.688578, 992.453539)],
 [(819.0, 861.0),
  (825.784385, 865.239354),
  (832.56877, 869.478708),
  (839.353154, 873.718062),
  (846.137539, 877.957416)],
 [(460.0, 375.0),
  (459.101206, 374.561629),
  (458.202412, 374.123258),
  (457.303618, 373.684887),
  (456.404824, 373.246515)],
 [(188.0, 896.0),
  (190.624236, 899.018838),
  (193.248472, 902.037677),
  (195.872708, 905.056515),
  (198.496944, 908.075353)],
 [(932, 11), (936, 11), (940, 11), (944, 11), (948, 11)],
 [(621.0, 610.0),
  (620.792088, 610.978148),
  (620.584177, 611.956295),
  (620.376265, 612.934443),
  (620.168353, 613.91259)],
 [(233.0, 228.0),
  (240.878462, 229.389185),
  (248.756924, 230.778371),
  (256.635386, 232.167556),
 

In [6]:
norm_all_data = df.normalize_data(trajectories)
train, test = df.split_test_train(norm_all_data)
train_data, train_masked_point = df.mask_point_at_index(train, 2)
test_data, test_masked_point = df.mask_point_at_index(test, 2)

print(len(train_data))
print(len(test_masked_point))

900
100


In [8]:
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)

    # ## DEBUG
print("src_data_tensor shape:", src_data_tensor.shape)
# print("src_data_tensor value:", src_data_tensor)
print("tgt_data_tensor shape:", tgt_data_tensor.shape)
# print("tgt_data_tensor value:", tgt_data_tensor)
print("src_masks_tensor shape:", src_masks_tensor.shape)

src_data_tensor shape: torch.Size([900, 5, 2])
tgt_data_tensor shape: torch.Size([900, 5, 2])
src_masks_tensor shape: torch.Size([900, 5])


In [12]:
import model as md

input_dim = 2
hidden_dim = 32
output_dim = 2
rate = 0.01

model = md.BasicTrajectoryModel(input_dim, hidden_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)


In [13]:
dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [14]:
for epoch in range(2):  # Adjust the number of epochs as needed
        model.train()
        total_loss = 0

        for src, tgt, mask in dataloader:
            optimizer.zero_grad()
            # Predict the output for the entire batch
            output = model(src.view(-1, input_dim))  # Reshape input to match model's expected shape
            output = output.view(src.shape)  # Reshape output to match target's shape
            
            # Compute loss only for the masked positions
            masked_output = output[mask]
            masked_tgt = tgt[mask]
            loss = loss_function(masked_output, masked_tgt)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 0.09150362631370282
Epoch 2, Loss: 0.07556695144238143
